In [1]:
from urllib import parse, request
from json import loads
import pandas as pd
import psycopg2 
import numpy as np

In [2]:
import csv

#Get categories mapping, index to category name
cats_map=[]
with open('numcats.txt','r') as f:
    reader=csv.reader(f,delimiter='\t')
    for num,cat in reader:
        cats_map.append(cat)

In [3]:
#Get books mapping, index to asin
asin = np.load('asin.npy')

# Solr

In [ ]:
#from urllib2 import *

In [ ]:
#connection = urlopen('http://localhost:8983/solr/bookstore/select?q=reviewText:good&wt=json')
#response = eval(connection.read())

In [ ]:
#response

# Asterixdb

In [6]:
class QueryResponse:
    def __init__(self, raw_response):
        self._json = loads(raw_response)

        self.requestID = self._json['requestID'] if 'requestID' in self._json else None
        self.clientContextID = self._json['clientContextID'] if 'clientContextID' in self._json else None
        self.signature = self._json['signature'] if 'signature' in self._json else None
        self.results = self._json['results'] if 'results' in self. _json else None
        self.metrics = self._json['metrics'] if 'metrics' in self._json else None

class AsterixConnection:
    def __init__(self, server = 'http://localhost', port = 19002):
        self._server = server
        self._port = port
        self._url_base = self._server +':'+ str(port)

    def query(self, statement, pretty=False, client_context_id=None):
        endpoint = '/query/service'

        url = self._url_base + endpoint

        payload = {
            'statement': statement,
            'pretty': pretty
        }

        if client_context_id:
            payload['client_context_id'] = client_context_id

        data = parse.urlencode(payload).encode("utf-8")
        req = request.Request(url, data)
        response = request.urlopen(req).read()

        return QueryResponse(response)

In [ ]:
#Get category information, grouped by nodeID
asterix_conn = AsterixConnection()
response = asterix_conn.query('''USE TinySocial;
    SELECT 
        nodeID, 
        (SELECT gp.cat.category.nested.level_1, gp.cat.category.nested.nested.level_2, gp.cat.category.nested.nested.nested.level_3, gp.cat.category.nested.nested.nested.nested.level_4, gp.cat.category.nested.nested.nested.nested.nested.level_5 FROM gp) AS categories
    FROM ClassificationInfo c
    GROUP BY c.nodeID AS nodeID
    GROUP AS gp(c as cat);''')

#print(response.results)

In [ ]:
#Parse result from asterixdb and create a dictionary {nodeID: [[lvl1,lvl2,lvl3,lvl4,lvl5],[lvl1,lvl2,..],..]}
max_cat = 0
categories = []

for i in range(len(response.results)):
    if len(response.results[i]['categories']) > max_cat:
        max_cat = len(response.results[i]['categories'])
    c = []
    for x in range(len(response.results[i]['categories'])):
        c.append([cats_map.index(response.results[i]['categories'][x][l]) for l in response.results[i]['categories'][x]])
    categories.append([response.results[i]['nodeID'],c])
    
categories = {i[0]:i[1] + [[0,0,0,0,0]]*(max_cat-len(i[1])) for i in categories}

Attempt to generate review features from Asterixdb but too slow

In [ ]:
#asterix_conn = AsterixConnection()
#response = asterix_conn.query('''  USE TinySocial;

#    SELECT 
#          r.overall as rating, 
#          r.helpful as votesForReview,
#          (SELECT count(rr)
#            FROM reviews rr
#           WHERE rr.reviewerID = r.reviewerID
#           ) as reviewerCount,
#          (SELECT count(br)
#            FROM reviews br
#           WHERE br.asin = r.asin
#           ) as bookreviewCount,
#          ltrim(substring_before(substring_after(r.helpful, ","),"]")) as outOf,
#          r.reviewTime
#    FROM reviews r
#    WHERE r.asin = "0000013714"
#limit 10;''')

#print(response.results)

# Postgres

In [19]:
conn = psycopg2.connect("dbname='SQLBook' user='postgres' host='localhost' password='1234'")
cur = conn.cursor()

In [5]:
cur.execute("""SELECT 
      r.reviewID,
      r.asin, 
      cast(r.overall as int) as rating, 
      cast(trim(leading '[' from substring(helpful from 0 for position(',' in helpful))) as int) as votesForReview,
      cast(trim(trailing ']' from substring(helpful from position(',' in helpful)+2)) as int) as outOf,
      reviewCount.cnt as reviewerCount,
      bookCount.cnt as bookreviewCount,
      EXTRACT(day from current_date - r.reviewTime) age,
      pr.nodeID
FROM reviews r
LEFT OUTER JOIN (SELECT reviewID, count(*) cnt FROM reviews GROUP BY reviewID) reviewCount ON r.reviewID = reviewCount.reviewID
LEFT OUTER JOIN (SELECT asin, count(*) cnt FROM reviews GROUP BY asin) bookCount ON r.asin = bookCount.asin
LEFT OUTER JOIN Products pr ON r.asin = pr.asin""")
rows = cur.fetchall()

In [ ]:
reviewFeatures = pd.DataFrame(rows,columns=['reviewID','asin','rating','helpfulness','outOf','reviewerCount','bokReviewCount','reviewAge','nodeID'])

# Categories Matrix

In [ ]:
#Get nodeid to asin mapping
cur.execute("""select nodeid,asin from products""")
rows = cur.fetchall()
nodeid_asin = pd.DataFrame(rows,columns=['nodeid','asin'])
nodeid_asin = nodeid_asin.set_index('asin')

In [ ]:
#Reorder categories with the nodeid to asin mapping (from postgres) and index to asin mapping (from asin.npy)
categories_indexed = []

for i in range(max_cat):
    c = []
    for a in asin:
        c.append(categories[int(nodeid_asin.ix[a.decode()]['nodeid'])][i])
    categories_indexed.append(c)

In [24]:
print(np.array(categories_indexed).shape)
np.array(categories_indexed)

(7, 3990, 5)


# Get seasonal information

In [54]:
#cur.execute("""select 
#	p.asin, 
#    ROUND(100*SUM(case when date_part('month', ol.billdate) between 7 and 9 then ol.numunits else 0 END)/CAST(SUM(ol.numunits) AS DECIMAL),2) summer,
#    ROUND(100*SUM(case when date_part('month', ol.billdate) between 10 and 12 then ol.numunits else 0 END)/CAST(SUM(ol.numunits) AS DECIMAL),2) fall,
#    ROUND(100*SUM(case when date_part('month', ol.billdate) between 1 and 3 then ol.numunits else 0 END)/CAST(SUM(ol.numunits) AS DECIMAL),2) winter,
#    ROUND(100*SUM(case when date_part('month', ol.billdate) between 4 and 6 then ol.numunits else 0 END)/CAST(SUM(ol.numunits) AS DECIMAL),2) spring
#from
#	orderlines ol
#    inner join products p on ol.productid = p.productid
#where
#    ol.numunits <> 0
#group by 
#    p.asin""")
#rows = cur.fetchall()

In [24]:
cur.execute("""select asin,
          round(100*sum(case when month >= 3 and month < 6 then numunits else 0 end)/sum(numunits),2) as spring,
          round(100*sum(case when month >= 6 and month < 9 then numunits else 0 end)/sum(numunits),2) as summer,
          round(100*sum(case when month >= 9 and month < 12 then numunits else 0 end)/sum(numunits),2) as fall,
          round(100*sum(case when (month = 12 or month < 3) then numunits else 0 end)/sum(numunits),2) as winter,
          cast(max(fullprice) as decimal) fullprice,
          case when max(isinstock) = 'Y' then 1 else 0 end isinstock
          from 
          (select asin, EXTRACT(MONTH FROM orderdate) as month, case when l.numunits = 0 then 0.00001 else l.numunits end as numunits, fullprice, isinstock
          from customers c, orders o, orderlines l, products p
          where c.customerid = o.customerid
          and o.orderid = l.orderid
          and l.productid = p.productid
          ) as temp
          group by asin""")
# """
rows = cur.fetchall()

In [25]:
season = pd.DataFrame(rows,columns=['asin','spring','summer','fall','winter','fullprice','isinstock'])

In [26]:
asin_season = []

for i in asin:
    if i.decode() in season.asin.values:
        asin_season.append([i for i in season[season.asin==i.decode()][['spring','summer','fall','winter','fullprice','isinstock']].values[0]])
    else:
        asin_season.append([0,0,0,0,0,''])

In [33]:
#spring, summer, fall, winter, fullprice, isinstock (1=yes, 0=no)
asin_season[0:5]

[[Decimal('0.00'),
  Decimal('33.33'),
  Decimal('0.00'),
  Decimal('66.67'),
  Decimal('540.00'),
  1],
 [Decimal('0.00'),
  Decimal('33.33'),
  Decimal('33.33'),
  Decimal('33.33'),
  Decimal('306.00'),
  1],
 [Decimal('25.00'),
  Decimal('0.00'),
  Decimal('25.00'),
  Decimal('50.00'),
  Decimal('340.00'),
  1],
 [Decimal('0.00'),
  Decimal('100.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('375.00'),
  1],
 [Decimal('0.00'),
  Decimal('100.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('540.00'),
  1]]

# Output Data

In [ ]:
#categories: dataframe that contains categories information from Asterixdb (nodeId and 5 levels of categories)
categories_indexed.to_csv('categories.csv')

#reviewFeatures: dataframe that contains review features from Postgres
reviewFeatures.to_csv('reviewFeatures.csv')

In [23]:
np.save('categories_indexed.npy',np.array(categories_indexed))